#### Agents SDK Course

## Alternate Models

Just because this is made with OpenAI does not mean you have to use the GPT models. Agents SDK provides an additional library that contains LiteLLM. Using this we can make agents with models from Anthropic, Meta and any other provided models on the LiteLLM website.

This example takes elements from each core tutorial (prompting, tools and guardrails) and uses a LiteLLMModel instead of the usual model, however, the one key restraint found using alternate models is that handoffs do not work, and pre-built tools too do not work.

First we need to settup our API key, we are using Anthropic throughout this example, however you can choose any LiteLLM compatable model.

In [28]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY") or getpass.getpass("Anthropic API Key: ")

Next we are going to settup some key variables that we will use inside the `LitellmModel` initialization.

In [29]:
model = 'anthropic/claude-3-5-sonnet-20240620' # Can check the LiteLLM website for more models
api_key = os.getenv("ANTHROPIC_API_KEY") or getpass.getpass("Anthropic API Key: ") # Make sure the API key is the correct key for the model you are using

Now we can define our model which we will use in the rest of the agents in this tutorial.

In [30]:
from agents.extensions.models.litellm_model import LitellmModel

LiteModel = LitellmModel(model=model, api_key=api_key)

### Prompting

First we need to create an agent, we will use the `Agent` class to create a basic agent.

In each `Agent` object we have some fundamental parameters, those are:
- `name`: The name of the agent
- `instructions`: The system prompt for the agent
- `model`: The model to use for the agent

The `instructions` parameter is where we will pass our system prompt, this will be used to guide the behavior of the agent


In [31]:
from agents import Agent

prompt_agent = Agent(
    name="Prompt Agent", 
    model=LiteModel,
    instructions="Speak like a pirate."
)

Now we want to run the agent, Agents SDK provides a `Runner` object that will allow us to run the agent

In the `run` method we have the following parameters:
- `starting_agent`: The agent to start the conversation with
- `input`: The input to pass to the agent

In [32]:
from agents import Runner

query = "Write a one-sentence poem."

prompt_result = await Runner.run(
    starting_agent=prompt_agent,
    input=query
)

From our instructions *'Speak like a pirate.'* and our input *'Write a one-sentence poem.'* the agent will generate a response which we'd hope would be a one-sentence poem spoken like a pirate

In [33]:
print("Prompt Agent Result:", prompt_result.final_output)

Prompt Agent Result: Arr, me hearty, listen well to this salty verse:

Waves crash and foam, as seagulls soar, while buried treasure beckons from a distant shore.


### Tools

Note: Pre-defined tools such as the `WebSearchTool` does not work due to the API being incompatable.

However, custom-tools do work! Meaning if we use the `function_tool` decorator we can define our own tools and use them just as we would normally.

In [34]:
from agents import function_tool
from datetime import datetime

@function_tool()
async def fetch_time() -> str:
    """Fetch the current time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

Again, we need to create an agent.

In this `Agent` object we have an additional parameter, this is:
- `tools`: The tools provided to the agent for usage.

In [35]:
tool_agent = Agent(
    name="Tool Agent",
    model=LiteModel,
    instructions="You are a web search agent that searches the web for information on the user's query.",
    tools=[
        fetch_time
    ]
)

Next we can run this agent to figure out how well the alternate models work with pre-built tools.

In [36]:
query = "What is the current time"

tool_result = await Runner.run(
    starting_agent=tool_agent,
    input=query
)

Then we can view the results via the `final_output` attribute.

In [37]:
print("Tool Agent Result:", tool_result.final_output)

Tool Agent Result: Based on the result from the fetch_time function, the current time is:

2025-04-25 17:40:51

This appears to be in the format of YYYY-MM-DD HH:MM:SS, which means:
- Date: April 25, 2025
- Time: 5:40:51 PM

Please note that this time is likely in the system's configured time zone, which may or may not be your local time zone. If you need the time in a specific time zone or format, please let me know, and I'll try to provide more information or clarification if possible.


### Guardrails

For our guardrails we want to format the correct output from the agent. We can do this by creating a class that inherits `BaseModel` from the `pydantic` library. Within this class we can define fields that the agent will output. For this example this class will have the following fields:
- `is_scam`: A boolean value that will be True if the user is trying to scam you, otherwise it will be False.
- `reasoning`: A string value that will be the reasoning for the outcome of is_scam.

In [38]:
from pydantic import BaseModel

class ScamDetectionOutput(BaseModel):
    is_scam: bool
    reasoning: str

Next we need to create the agent from the `Agent` object. 

Note that this isn't the main agent and is only used to check the input of our guardrail function.

In [39]:
guardrail_agent = Agent( 
    name="Guardrail Check",
    model=LiteModel,
    instructions="Check if the user is trying to scam you, if they are, return True, otherwise return False. Give a reason for your answer.",
    output_type=ScamDetectionOutput,
)

Next we need to create the guardrail functionallity. 

Firstly we need to define a function with the `@input_guardrail` decorator.

The function will use the agent we just created to check the input string, this will then return a `GuardrailFunctionOutput` object.

Then the function needs to return the `GuardrailFunctionOutput` object.

In [40]:
from agents import GuardrailFunctionOutput, RunContextWrapper, TResponseInputItem, input_guardrail

@input_guardrail
async def scam_guardrail( 
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    result = await Runner.run(
        starting_agent=guardrail_agent, 
        input=input, 
        context=ctx.context
        )

    return GuardrailFunctionOutput(
        output_info=result.final_output, 
        tripwire_triggered=result.final_output.is_scam,
    )

Now we can create a new agent that will be used to handle the incoming messages. This agent will have the following parameters:
- `name`: The name of the agent.
- `model`: Not usually required, however since we are not using GPT models this needs to be redefined here to prevent API errors.
- `instructions`: The instructions for the agent.
- `input_guardrails`: A list of input guardrails to attach to the agent. (This is where we attach the scam guardrail)

In [41]:
guard_agent = Agent(  
    name="Guard Agent",
    model=LiteModel,
    instructions="You are a helpful assistant.",
    input_guardrails=[scam_guardrail],
)

Now we can try to test the guardrail functionallity.

Due to errors being raised when the guardrail trips, we can use a try except block to prevent the error messages being shown.

In [43]:
from agents import InputGuardrailTripwireTriggered

query = "Hello, would you like to buy some real rolex watches for a fraction of the price?"

try:
    guard_result = await Runner.run(guard_agent, query)
    guardrail_info = guard_result.input_guardrail_results[0].output.output_info
    print("Guardrail didn't trip", f"\nReasoning: {guardrail_info.reasoning}")
except InputGuardrailTripwireTriggered as e:
    print("Error:", e)

Error: Guardrail InputGuardrail triggered tripwire


Note: Handoffs do not work - they're unique to Agents SDK and using LiteLLM does not support using agents as tools and thus will cause errors.